In [0]:
!pip install tensorly

In [0]:
import keras
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.models import Sequential
from keras.optimizers import SGD
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from sklearn import datasets
import numpy as np
import tensorly as tl
from tensorly.decomposition import parafac
from tensorly.decomposition import tucker
import matplotlib.pyplot as pl

In [0]:
from sklearn.datasets import fetch_openml
dataset = fetch_openml('mnist_784')

In [0]:
# Reshape the data to a (70000, 28, 28) tensor
data = dataset.data.reshape((dataset.data.shape[0], 28, 28))

In [0]:
# Reshape the data to a (70000, 28, 28, 1) tensord
data = data[:, :, :, np.newaxis]

In [0]:
# Scale values from range of [0-255] to [0-1]
scaled_data = data / 255.0

In [0]:
# Split the dataset into training and test sets
(train_data, test_data, train_labels, test_labels) = train_test_split(
    scaled_data,
    dataset.target.astype("int"), 
    test_size = 0.33)


In [0]:
# Tranform training labels to one-hot encoding
train_labels = np_utils.to_categorical(train_labels, 10)

In [0]:
# Tranform test labels to one-hot encoding
test_labels = np_utils.to_categorical(test_labels, 10)

In [0]:
# Create a sequential model
model = Sequential()

# Add the first convolution layer
model.add(Convolution2D(
    filters = 20,
    kernel_size = (5, 5),
    padding = "same",
    input_shape = (28, 28, 1)))

# Add a ReLU activation function
model.add(Activation(
    activation = "relu"))

# Add a pooling layer
model.add(MaxPooling2D(
    pool_size = (2, 2),
    strides =  (2, 2)))

# Add the second convolution layer
model.add(Convolution2D(
    filters = 50,
    kernel_size = (5, 5),
    padding = "same"))

# Add a ReLU activation function
model.add(Activation(
    activation = "relu"))

# Add a second pooling layer
model.add(MaxPooling2D(
    pool_size = (2, 2),
    strides = (2, 2)))

# Flatten the network
model.add(Flatten())

# Add a fully-connected hidden layer
model.add(Dense(500))

# Add a ReLU activation function
model.add(Activation(
    activation = "relu"))

# Add a fully-connected output layer
model.add(Dense(10))

# Add a softmax activation function
model.add(Activation("softmax"))

In [0]:
model.summary()

In [0]:
# Compile the network
model.compile(
    loss = "categorical_crossentropy", 
    optimizer = SGD(lr = 0.01),
    metrics = ["accuracy"])
#print('\n layer_1 веса до обучения :\n', model.layers[0].weights[0])
print(model.layers[0].weights[0].shape)
print(model.layers[3].weights[0].shape)

In [0]:
epochs = 20

# Train the model 
history = model.fit(
    train_data, 
    train_labels, 
    batch_size = 128, 
    epochs = epochs,
	  verbose = 1)


In [0]:
# Evaluate the model
(loss, accuracy) = model.evaluate(
    test_data, 
    test_labels,
    batch_size = 128, 
    verbose = 1)

In [0]:
# Print the model's accuracy
print(accuracy)
print(loss)

In [0]:
acc = history.history['accuracy']

loss = history.history['loss']

epochs_range = range(epochs)

plt.figure(figsize=(10,10))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.legend(loc='lower right')
plt.title('Training Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.legend(loc='upper right')
plt.title('Training Loss')
plt.show()

In [0]:
def decomposition(layer, ranks):
  '''takes a convolutional layer, prepare tucker decomposition on its weights
  and return a tucker_to_tensor object with biases'''
  old_weights = layer.weights[0].numpy()
  biases = layer.weights[1].numpy()
  core, factors = tucker(old_weights, ranks = ranks)
  new_weights = tl.tucker_to_tensor((core, factors))
  print('Well done Tucker decomposition, residual is: \n', tl.norm(new_weights-old_weights))
  return [new_weights, biases]

In [0]:
def parafac_decomposition(layer, rank):
  '''takes a convolutional layer, prepare PARAFAC decomposition on its weights
  and return a tucker_to_tensor object with biases'''
  old_weights = layer.weights[0].numpy()
  biases = layer.weights[1].numpy()
  core, factors = parafac(old_weights, rank, init = 'svd')
  new_weights = tl.kruskal_to_tensor((core, factors))
  print('Well done parafac decomposition, residual is: \n', tl.norm(new_weights-old_weights))
  return [new_weights, biases]

In [0]:
import copy
model_tucker = copy.deepcopy(model)
model_parafac = copy.deepcopy(model)

'''experiment 1'''
model_tucker.layers[0].set_weights(
    decomposition(model.layers[0], ranks = (3, 3, 1, 10)))
model_tucker.layers[3].set_weights(
    decomposition(model.layers[3], ranks = (3, 3, 10, 25)))
(loss_t, accuracy_t) = model_tucker.evaluate(
    test_data, 
    test_labels,
    batch_size = 128, 
    verbose = 1)
print('Evaluation done!')
print('Accuracy and loss with Tucker Decomposition : \n', accuracy_t, loss_t)
print('Difference in accuracy : \n', accuracy-accuracy_t)
print('\n')

'''experiment 2'''
model_parafac.layers[0].set_weights(
    parafac_decomposition(model.layers[0], rank = 1))
model_parafac.layers[3].set_weights(
    parafac_decomposition(model.layers[3], rank = 6))
(loss_p, accuracy_p) = model_parafac.evaluate(
    test_data, 
    test_labels,
    batch_size = 128, 
    verbose = 1)
print('Evaluation done!')
print('Accuracy and loss with Parafac Decomposition : \n', accuracy_p, loss_p)
print('Difference in accuracy : \n', accuracy-accuracy_p)

In [0]:
model_tucker = copy.deepcopy(model)
model_parafac = copy.deepcopy(model)

'''experiment 3'''
model_tucker.layers[0].set_weights(
    decomposition(model.layers[0], ranks = (5, 5, 1, 20)))
model_tucker.layers[3].set_weights(
    decomposition(model.layers[3], ranks = (5, 5, 20, 50)))
(loss_t, accuracy_t) = model_tucker.evaluate(
    test_data, 
    test_labels,
    batch_size = 128, 
    verbose = 1)
print('Evaluation done!')
print('Accuracy and loss with Tucker Decomposition : \n', accuracy_t, loss_t)
print('Difference in accuracy : \n', accuracy-accuracy_t)
print('\n')

'''experiment 4'''
model_parafac.layers[0].set_weights(
    parafac_decomposition(model.layers[0], rank = 20))
model_parafac.layers[3].set_weights(
    parafac_decomposition(model.layers[3], rank = 500))
(loss_p, accuracy_p) = model_parafac.evaluate(
    test_data, 
    test_labels,
    batch_size = 128, 
    verbose = 1)
print('Evaluation done!')
print('Accuracy and loss with Parafac Decomposition : \n', accuracy_p, loss_p)
print('Difference in accuracy : \n', accuracy-accuracy_p)

In [0]:
model_tucker = copy.deepcopy(model)
model_parafac = copy.deepcopy(model)

'''experiment 5'''
model_tucker.layers[0].set_weights(
    decomposition(model.layers[0], ranks = (1, 1, 1, 5)))
model_tucker.layers[3].set_weights(
    decomposition(model.layers[3], ranks = (2, 2, 5, 15)))
(loss_t, accuracy_t) = model_tucker.evaluate(
    test_data, 
    test_labels,
    batch_size = 128, 
    verbose = 1)
print('Evaluation done!')
print('Accuracy and loss with Tucker Decomposition : \n', accuracy_t, loss_t)
print('Difference in accuracy : \n', accuracy-accuracy_t)
print('\n')

'''experiment 6'''
model_parafac.layers[0].set_weights(
    parafac_decomposition(model.layers[0], rank = 1))
model_parafac.layers[3].set_weights(
    parafac_decomposition(model.layers[3], rank = 6))
(loss_p, accuracy_p) = model_parafac.evaluate(
    test_data, 
    test_labels,
    batch_size = 128, 
    verbose = 1)
print('Evaluation done!')
print('Accuracy and loss with Parafac Decomposition : \n', accuracy_p, loss_p)
print('Difference in accuracy : \n', accuracy-accuracy_p)